# Binance IDO Token Performance Analysis
Analyzing the performance of Binance IDO tokens using candlestick patterns and price changes.


In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
import time

# List of Binance IDO tokens (keeping the list shorter for testing)
tokens = ['HOME', 'SKATE', 'SOON', 'EDGEN', 'FLY']

def get_klines_data(symbol):
    """Fetch 5-minute candlestick data for a token"""
    base_url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': f'{symbol}USDT',
        'interval': '5m',
        'limit': 288  # 24 hours of 5-minute candles
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if not data:
            print(f"No data received for {symbol}")
            return None

        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df.set_index('timestamp', inplace=True)
        return df

    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return None


/Users/ilya.misiukevich/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import requests
import pandas as pd
import mplfinance as mpf
from datetime import datetime, timedelta
import time

def get_mexc_klines(symbol, interval='5m', limit=288):
    """Fetch kline/candlestick data from MEXC spot market"""
    base_url = "https://api.mexc.com/api/v3/klines"

    params = {
        'symbol': f'{symbol}USDT',  # MEXC spot pairs don't use underscore
        'interval': interval,
        'limit': limit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received for {symbol}")
            return None

        # MEXC spot klines format:
        # [openTime, open, high, low, close, volume, closeTime, quoteVolume, trades, takerBuyBaseVol, takerBuyQuoteVol, ignore]
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df.set_index('timestamp', inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching {symbol} from MEXC: {str(e)}")
        return None

    # Current timestamp in milliseconds
    end_time = int(time.time() * 1000)
    # Calculate start time (24 hours ago)
    start_time = end_time - (24 * 60 * 60 * 1000)  # 24 hours in milliseconds

    params = {
        'symbol': f'{symbol}_USDT',
        'interval': interval,
        'start': start_time,
        'end': end_time
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }

    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()

        if 'data' not in data or not data['data']:
            print(f"No data received for {symbol}")
            return None

        # MEXC returns data in format [timestamp, open, close, high, low, volume, amount]
        klines = data['data']
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'close', 'high', 'low', 'volume', 'amount'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df.set_index('timestamp', inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching {symbol} from MEXC: {str(e)}")
        return None

# List of tokens on MEXC
tokens = ['HOME']

# Store performance metrics
performance_data = []

# Process each token
for token in tokens:
    print(f"Processing {token} from MEXC...")
    df = get_mexc_klines(token)

    if df is not None and not df.empty:
        # Calculate metrics before plotting
        initial_price = df['open'].iloc[0]
        current_price = df['close'].iloc[-1]
        price_change = ((current_price - initial_price) / initial_price) * 100

        # Create custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, 
                              gridstyle='',
                              y_on_right=False)

        # Create and show the plot
        mpf.plot(df, 
                type='candle',
                style=s,
                title=f'\n{token}/USDT on MEXC\nPrice Change: {price_change:.2f}%',
                volume=True,
                figsize=(12, 8),
                panel_ratios=(2, 1),
                ylabel='Price ($)',
                ylabel_lower='Volume',
                tight_layout=True)

        # Record metrics
        performance_data.append({
            'Token': token,
            'Exchange': 'MEXC',
            'Initial Price': initial_price,
            'Current Price': current_price,
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume (USDT)': df['volume'].sum(),
            'Change %': price_change
        })

    time.sleep(1)  # Rate limiting


Processing HOME from MEXC...
Error fetching HOME from MEXC: 12 columns passed, passed data had 8 columns


In [4]:
# Display performance summary
if performance_data:
    summary_df = pd.DataFrame(performance_data)
    summary_df = summary_df.sort_values('Change %', ascending=False)
    pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
    print("\nPerformance Summary on MEXC:")
    display(summary_df)
        'symbol': f'{symbol}_USDT',
        'interval': interval,
        'limit': limit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received for {symbol}")
            return None

        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df.set_index('timestamp', inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching {symbol} from MEXC: {str(e)}")
        return None

# List of tokens on MEXC
tokens = ['HOME']  # Add more tokens that are listed on MEXC

# Store performance metrics
performance_data = []

# Process each token
for token in tokens:
    print(f"Processing {token} from MEXC...")
    df = get_mexc_klines(token)

    if df is not None and not df.empty:
        # Calculate metrics before plotting
        initial_price = df['open'].iloc[0]
        current_price = df['close'].iloc[-1]
        price_change = ((current_price - initial_price) / initial_price) * 100

        # Create custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, 
                              gridstyle='',
                              y_on_right=False)

        # Create and show the plot
        mpf.plot(df, 
                type='candle',
                style=s,
                title=f'\n{token}/USDT on MEXC\nPrice Change: {price_change:.2f}%',
                volume=True,
                figsize=(12, 8),
                panel_ratios=(2, 1),
                ylabel='Price ($)',
                ylabel_lower='Volume',
                tight_layout=True)

        # Record metrics
        performance_data.append({
            'Token': token,
            'Exchange': 'MEXC',
            'Initial Price': initial_price,
            'Current Price': current_price,
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume (USDT)': df['volume'].sum(),
            'Change %': price_change
        })

    time.sleep(1)  # Rate limiting


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 65)

In [ ]:
# Display performance summary
if performance_data:
    summary_df = pd.DataFrame(performance_data)
    summary_df = summary_df.sort_values('Change %', ascending=False)
    pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
    print("\nPerformance Summary on MEXC:")
    display(summary_df)
        initial_price = df['open'].iloc[0]
        current_price = df['close'].iloc[-1]
        price_change = ((current_price - initial_price) / initial_price) * 100

        # Create custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, 
                              gridstyle='',
                              y_on_right=False)

        # Create and show the plot
        mpf.plot(df, 
                type='candle',
                style=s,
                title=f'\n{token}/USDT Performance\nPrice Change: {price_change:.2f}%',
                volume=True,
                figsize=(10, 6),
                panel_ratios=(2, 1),
                ylabel='Price ($)',
                ylabel_lower='Volume',
                tight_layout=True,
                warn_too_much_data=10000)

        # Record metrics
        performance_data.append({
            'Token': token,
            'Initial Price': initial_price,
            'Current Price': current_price,
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume (USDT)': df['volume'].sum(),
            'Change %': price_change
        })

    time.sleep(1)  # Rate limiting


In [ ]:
# Display performance summary
if performance_data:
    summary_df = pd.DataFrame(performance_data)
    summary_df = summary_df.sort_values('Change %', ascending=False)
    pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
    print("\nPerformance Summary (Sorted by Price Change %):")
    display(summary_df)
    print(f"Processing {token}...")
    df = get_klines_data(token)

    if df is not None and not df.empty:
        # Create subplot
        ax = plt.subplot(rows, 3, idx)

        # Calculate price change percentage
        initial_price = df['open'].iloc[0]
        current_price = df['close'].iloc[-1]
        price_change = ((current_price - initial_price) / initial_price) * 100

        # Create custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, 
                              gridstyle='',
                              y_on_right=False)

        # Plot candlesticks
        mpf.plot(df, 
                type='candle',
                style=s,
                title=f'{token}/USDT\nChange: {price_change:.2f}%',
                volume=True,
                panel_ratios=(2, 1),
                ylabel='Price',
                ylabel_lower='Volume',
                ax=ax)

        # Record metrics
        performance_data.append({
            'Token': token,
            'Initial Price': initial_price,
            'Current Price': current_price,
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume': df['volume'].sum(),
            'Change %': price_change
        })

    time.sleep(1)  # Rate limiting

plt.tight_layout()
plt.show()


In [ ]:
# Display performance summary
if performance_data:
    summary_df = pd.DataFrame(performance_data)
    summary_df = summary_df.sort_values('Change %', ascending=False)
    pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
    print("\nPerformance Summary (Sorted by Price Change %):")
    display(summary_df)
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error fetching {symbol}: {response.status_code}")
            return None

        data = response.json()
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Set timestamp as index for mplfinance
        df.set_index('timestamp', inplace=True)

        # Calculate percentage change
        df['change_pct'] = ((df['close'] - df['open'].iloc[0]) / df['open'].iloc[0]) * 100

        return df
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return None


In [ ]:
# Create a figure with subplots for candlestick charts
plt.style.use('dark_background')
rows = (len(tokens) + 2) // 3  # 3 columns
fig, axes = plt.subplots(rows, 3, figsize=(20, rows * 6))
fig.suptitle('First 24 Hours Trading Performance of Binance IDO Tokens (5min Candles)', fontsize=16)
axes = axes.flatten()

# Store performance metrics
performance_data = []

# Process each token
for idx, token in enumerate(tokens):
    print(f"Processing {token}...")
    df = get_klines_data(token)

    if df is not None and not df.empty:
        # Create individual figure for each token
        fig_token = mpf.figure(figsize=(6, 4))
        ax1 = fig_token.add_subplot(2, 1, 1)  # Price subplot
        ax2 = fig_token.add_subplot(2, 1, 2)  # Volume subplot

        # Create a custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, gridstyle='')


# Binance IDO Token Performance Analysis
Analysis of first 24 hours trading data using candlestick patterns


In [ ]:
import requests
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime
import time

# List of Binance IDO tokens
tokens = ['HOME', 'SKATE', 'SOON', 'EDGEN', 'FLY', 'CUDIS', 'BOMB', 'RESOLV', 'RDO', 'SERAPH']

def get_klines_data(symbol):
    """Fetch 5-minute candlestick data for first 24 hours"""
    base_url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': f'{symbol}USDT',
        'interval': '5m',
        'limit': 288  # 24 hours of 5-minute candles
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error fetching {symbol}: {response.status_code}")
            return None

        data = response.json()
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        # Convert types
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Set timestamp as index for mplfinance
        df.set_index('timestamp', inplace=True)

        return df
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return None


In [ ]:
# Process each token and create individual plots
performance_data = []

for token in tokens:
    print(f"Processing {token}...")
    df = get_klines_data(token)

    if df is not None and not df.empty:
        # Calculate initial metrics
        initial_price = df['open'].iloc[0]
        final_price = df['close'].iloc[-1]
        price_change = ((final_price - initial_price) / initial_price) * 100

        # Create custom style
        mc = mpf.make_marketcolors(up='g', down='r',
                                 edge='inherit',
                                 wick='inherit',
                                 volume='in',
                                 ohlc='inherit')
        s = mpf.make_mpf_style(marketcolors=mc, 
                              gridstyle='',
                              y_on_right=False)

        # Create figure and save it
        fig, axes = mpf.plot(df, 
                           type='candle',
                           title=f'\n{token}/USDT Performance\nChange: {price_change:.2f}%',
                           style=s,
                           volume=True,
                           panel_ratios=(2, 1),
                           ylabel='Price',
                           ylabel_lower='Volume',
                           returnfig=True)

        plt.close(fig)

        # Record performance metrics
        performance_data.append({
            'Token': token,
            'Initial Price': initial_price,
            'Final Price': final_price,
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume (USDT)': df['volume'].sum(),
            'Price Change %': price_change
        })

    time.sleep(0.5)  # Rate limiting


In [ ]:
# Create summary of performance
if performance_data:
    summary_df = pd.DataFrame(performance_data)
    summary_df = summary_df.sort_values('Price Change %', ascending=False)
    pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
    print("\nPerformance Summary (Sorted by Price Change %):")
    print(summary_df)


In [ ]:
# Display the top 5 performers and bottom 5 performers
print("\nTop 5 Performers:")
print(summary_df.head())
print("\nBottom 5 Performers:")
print(summary_df.tail())

        # Place the plot in the main figure grid
        fig.delaxes(axes[idx])
        fig.add_subplot(axes[idx])
        axes[idx].imshow(fig_token.canvas.renderer.buffer_rgba())
        axes[idx].axis('off')
        plt.close(fig_token)

        # Record performance metrics
        performance_data.append({
            'Token': token,
            'Initial Price': df['open'].iloc[0],
            'Final Price': df['close'].iloc[-1],
            'High': df['high'].max(),
            'Low': df['low'].min(),
            'Volume (USDT)': df['quote_volume'].sum(),
            'Price Change %': df['change_pct'].iloc[-1]
        })

    time.sleep(0.5)  # Rate limiting

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# Display performance summary
summary_df = pd.DataFrame(performance_data)
summary_df = summary_df.sort_values('Price Change %', ascending=False)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
print("\nPerformance Summary (Sorted by Price Change %):")
print(summary_df)
    """Get 5-minute klines data for first 24 hours"""
    url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': f'{symbol}USDT',
        'interval': '5m',
        'limit': 288  # 24 hours worth of 5-minute candles
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close',
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
        ])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)

        # Calculate percentage change from initial price
        initial_price = float(df['open'].iloc[0])
        df['price_change_pct'] = ((df['close'] - initial_price) / initial_price) * 100

        return df

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Create plot
plt.figure(figsize=(15, 12))

# Store performance data
performance_data = []

for i, token in enumerate(tokens, 1):
    print(f"Processing {token}...")
    df = get_token_data(token)

    if df is not None and not df.empty:
        plt.subplot(5, 2, i)
        plt.plot(df.index, df['price_change_pct'], label=f'{token}/USDT')

        # Add horizontal line at 0%
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.3)

        # Add title and labels
        plt.title(f'{token}/USDT Performance')
        plt.xlabel('Time (5min intervals)')
        plt.ylabel('Price Change %')

        # Add grid
        plt.grid(True, alpha=0.3)

        # Record performance metrics
        performance_data.append({
            'Token': token,
            'Initial Price': df['open'].iloc[0],
            'Final Price': df['close'].iloc[-1],
            'Max Change %': df['price_change_pct'].max(),
            'Min Change %': df['price_change_pct'].min(),
            'Final Change %': df['price_change_pct'].iloc[-1]
        })
